# Power Outages

**Name(s)**: Karsin Dass & Nicole Doyle

**Website Link**: (your website link)

In [2]:
import pandas as pd
import numpy as np

import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from lec_utils import * # Feel free to uncomment and use this. It'll make your plotly graphs look like ours in lecture!

## Step 1: Introduction

In [3]:
#Is there a specific day of the week and time of day where intentional attacks occur?  Do they occur more frequently in urban areas where more people are affected?

## Step 2: Data Cleaning and Exploratory Data Analysis

In [12]:
cleaned_data = pd.read_csv('outage.csv', skiprows=5)
cleaned_data = cleaned_data.dropna(how='all', axis=1)
cleaned_data = cleaned_data.dropna(how='all')
cleaned_data = cleaned_data.drop(0)
print(cleaned_data.head())



  variables  OBS    YEAR  MONTH U.S._STATE POSTAL.CODE NERC.REGION  \
1       NaN  1.0  2011.0    7.0  Minnesota          MN         MRO   
2       NaN  2.0  2014.0    5.0  Minnesota          MN         MRO   
3       NaN  3.0  2010.0   10.0  Minnesota          MN         MRO   
4       NaN  4.0  2012.0    6.0  Minnesota          MN         MRO   
5       NaN  5.0  2015.0    7.0  Minnesota          MN         MRO   

       CLIMATE.REGION ANOMALY.LEVEL CLIMATE.CATEGORY  ... POPPCT_URBAN  \
1  East North Central          -0.3           normal  ...        73.27   
2  East North Central          -0.1           normal  ...        73.27   
3  East North Central          -1.5             cold  ...        73.27   
4  East North Central          -0.1           normal  ...        73.27   
5  East North Central           1.2             warm  ...        73.27   

  POPPCT_UC POPDEN_URBAN POPDEN_UC POPDEN_RURAL AREAPCT_URBAN AREAPCT_UC  \
1     15.28         2279    1700.5         18.2          2

In [2]:
# TODO

## Step 3: Framing a Prediction Problem

In [ ]:
# TODO

## Step 4: Baseline Model

In [ ]:
# TODO

## Step 5: Final Model

In [ ]:
# TODO